# **Bird Population Regression Analysis** #
### _Linear regression analysis of bird popualtion data including log transformed populations and analysis of single species._ ###
### _Uses 'Occurance and Climate Data of Birds in Asia' Dataset by Prathamesh Patil 2025._ ###

In [ ]:
# Step 1: Import libraries
#-------------------------

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Step 2: Load the Dataset
#-------------------------

df = pd.read_csv("Occurance_and_climatedata_of_birds.csv") # Dataset URL in README file

# Overview of the data
print(df.head())
print(df.info())
print(df.describe())

#Check for missing values
print(df.isnull().sum())


In [ ]:
# Step 3: Clean the Data
#-----------------------

# Drop 'Shift_km' column to avoid potential multicollinearity (the 'Shift_km' variable may have been calculated from other variables in model)
df_clean = df.drop(columns=['Shift_km'])

# Drop rows with missing values
df_clean = df_clean.dropna()


In [ ]:
# Step 4: a) Run Linear regression (untransformed population) b) Evaluate Model
# -----------------------------------------------------------------------------

# a) Run Linear Regression

print("\n=== Linear Regression (Raw Population) ===")

# Define the predictors and response
X = df_clean[['Temperature', 'Precipitation', 'Traffic', 'Latitude', 'Longitude', 'Year']]
y = df_clean['Population']

# Split into training and test sets (80:20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the linear regression
model = LinearRegression()
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

#------------------------------------------------------------------------------------------------

# b) Evaluate model perfromance

r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"R²: {r2}") # how much of the variance in the dependent variable the model explains
print(f"RMSE: {rmse}") # the average size of the errors between the predicted and observed values

y.mean(), y.median(), y.min(), y.max() # Comapre to R² and RMSE values

# average prediction error is ~60% of the mean population
# the model is not capturing population patterns well

# transforming the data may reduce skew, stabilise variance etc.
# log transform the population data to better meet the assumption of the linear model

In [ ]:
# Step 5: a) Log-transform the population and re-run regression b) Evaluate Model
# -------------------------------------------------------------------------------

# a) Log-transform the population and re-run regression

print("\n=== Linear Regression (Log-Transformed Population) ===")

# Add log-transformed population
df_clean['log_population'] = np.log1p(df_clean['Population'])  # log(1 + x) avoids issues with zeros

# Define response as log-transformed population
y_log = df_clean['log_population']

# Split into training and test sets (80:20)
X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=0.2, random_state=42)

# Fit linear regression
model_log = LinearRegression()
model_log.fit(X_train, y_train)

# Predict on test set
y_pred_log = model_log.predict(X_test)

#----------------------------------------------------------------------------------------------------------------

# b) Evaluate model perfromance

r2_log = r2_score(y_test, y_pred_log)
rmse_log = np.sqrt(mean_squared_error(y_test, y_pred_log))

print(f"R²: {r2_log}") # how much of the variance in the dependent variable the model explains
print(f"RMSE (log scale): {rmse_log}") # the average size of the errors between the predicted and observed values

# log-transforming helped stabilise the scale, but did not uncover a strong linear relationship
# a log error of ~0.9 corresponds to a predicting population error of a factor of ~2.5
# this is typical of ecological data- however the model is still weak

# bird population counts include common and rare species from different niches- this could be affecting the result
# proceed with log transformed population for individual bird species

In [ ]:
# Step 6: a) Run Analysis of single species with log transformed population b) Evaluate Model
# ---------------------------------------------------------------

# a) Run Analysis of single species with log-transformed population

print("\n=== Single-Species Linear Regression (Log Population) ===")

# Check species counts and choose species with high nember of counts
print(df_clean['Bird_Species'].value_counts().head())
species_name = 'Bar-headed Goose'  # In this case the Bar-headed Goose had 193 counts- I have chosen this species to test the regression
df_sp = df_clean[df_clean['Bird_Species'] == species_name].copy()

# Define predictors and response
X_sp = df_sp[['Temperature', 'Precipitation', 'Traffic', 'Latitude', 'Longitude', 'Year']]
y_sp = df_sp['log_population']

# Split into training and test sets (80:20)
X_train, X_test, y_train, y_test = train_test_split(X_sp, y_sp, test_size=0.2, random_state=42)

# Fit the linear regression
model_sp = LinearRegression()
model_sp.fit(X_train, y_train)

# Predict on test set
y_pred_sp = model_sp.predict(X_test)

#----------------------------------------------------------------------------------------------

# b) Evaluate performance

r2_sp = r2_score(y_test, y_pred_sp)
rmse_sp = np.sqrt(mean_squared_error(y_test, y_pred_sp))

print(f"R²: {r2_sp}")
print(f"RMSE (log scale): {rmse_sp}")
# Even within one species, population size is not well explained by the available predictors
# Linear relationships are extremely weak or absent

# Inspect coefficients
coeffs = pd.DataFrame({
    'Variable': X_sp.columns,
    'Coefficient': model_sp.coef_
})
print("\nCoefficients:\n", coeffs)
# These show the expected change in log(population) for a one-unit increase in the predictors
# The coefficent values demonstrate small or even negligible effects

# Plot observed vs predicted
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred_sp, alpha=0.5)
plt.plot([y_test.min(), y_test.max()],
         [y_test.min(), y_test.max()], 'r--')
plt.xlabel("Observed log(Population)")
plt.ylabel("Predicted log(Population)")
plt.title(f"Observed vs Predicted (Log Population) - {species_name}")
plt.show()
# This plot helps demonstrate that the model is not a good fit